### 0. 라이브러리

In [1]:
import os

# OS 충돌 방지
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Matplotlib 커널 종료 방지
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [25]:
# 라이브러리
import warnings
warnings.filterwarnings('ignore')

import time
import random
import pickle

import pandas as pd
import numpy as np

from tqdm import tqdm

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [4]:
# 시드 고정
def fixSEED(seed, deterministic=True):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    if deterministic:
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

seed = 42
fixSEED(seed=seed)

In [5]:
dir = '/Summary'
path = '/Users/leesanghyuk/Python_Programs/Projects/Models/' + dir
os.chdir(path)
print('Current Directory:', os.getcwd())

Current Directory: /Users/leesanghyuk/Python_Programs/Projects/Models/Summary


In [6]:
# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [7]:
# 학습 시작 전 GPU 메모리 확인
def checkGPU(empty=False):
    if torch.cuda.is_available():
        total_mem = torch.cuda.get_device_properties(0).total_memory
        current_mem = torch.cuda.memory_allocated(0)
        free_mem = total_mem - current_mem
        
        print(f'남은 GPU 메모리: {free_mem / (1024 ** 3):.2f} GB')
        
        if empty:
            torch.cuda.empty_cache()
            after_mem = torch.cuda.memory_allocated(0)
            print(f'GPU 캐시에서 메모리 삭제 후 메모리: {after_mem / (1024 ** 3):.2f} GB')
    else:
        print('CUDA 사용 불가')

### 1. Summary 모델 정의

In [26]:
class KoBARTSummary:
    def __init__(self,
                 text,
                 model_checkpoint = 'gogamza/kobart-summarization', 
                 tokenizer_checkpoint = 'gogamza/kobart-summarization', 
                 max_length=512, 
                 device='cpu'):
        self.text = text
        self.model = BartForConditionalGeneration.from_pretrained(model_checkpoint).to(device)
        self.tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_checkpoint)
        self.max_length = max_length
        self.device = device
    
    def getSummary(self):
        raw_input_ids = self.tokenizer.encode(self.text)
        input_ids = [self.tokenizer.bos_token_id] + raw_input_ids + [self.tokenizer.eos_token_id]
        
        # 입력 길이 제한 초과는 제거
        if len(input_ids) > self.max_length:
            input_ids = input_ids[:self.max_length - 1] + [self.tokenizer.eos_token_id]
        
        summary_ids = self.model.generate(torch.tensor([input_ids]), max_length=self.max_length)
        result = self.tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

        return result
        

### 2. 데이터 불러오기

In [27]:
# 데이터 불러오기
with open('/Users/leesanghyuk/Python_Programs/Projects/Data/Youtube_Image/youtubeData(8331)_with_label.pkl', 'rb') as f:
    data = pickle.load(f)

In [29]:
# 20개 샘플링
sampled_data = data.sample(20, random_state=seed)

In [30]:
sampled_data['label'].value_counts()

label
0    12
1     8
Name: count, dtype: int64

### 3. 요약문 생성하기

In [31]:
# 요약문 저장 열 생성
sampled_data['summary'] = ''


# 각 데이터에 대해 요약문 생성
for i in tqdm(sampled_data.index, desc='Creating Summary'):
    text = sampled_data.loc[i, 'content']
    model = KoBARTSummary(text)
    summary_text = model.getSummary()
    sampled_data.loc[i, 'summary'] = summary_text

Creating Summary:   0%|          | 0/20 [00:00<?, ?it/s]You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
Creating Summary:   5%|▌         | 1/20 [00:03<01:07,  3.55s/it]You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels 

### 4. 결과 확인

In [32]:
# 완료 파일 확인
def check_samples(data):
    
    # 하나씩 변경 확인을 위해 출력
    for idx, row in data.iterrows():
        print('-'*50)
        print(f"Sample {idx}")
        print('\n🔶 영상 요약 전')
        print(data.loc[idx, 'content'])
        print('\n🔷 영상 요약 전 후')
        print(data.loc[idx, 'summary'])
        print('')

In [33]:
# 확인
check_samples(sampled_data)

--------------------------------------------------
Sample 7366

🔶 영상 요약 전
안녕하세요 ooo 참이 입니다 아 아 자 이번 알리익스프레스 아우른 집 꽃 템 스페셜로 꾸며봤습니다 2 이번 영상을  백과 함께 만든 영상 이거든요 도움을 쓰고 돈을 또 받는법 궁금하시다면 묶어서 보십시오 첫번째 아이템 자 꽃병을 살려고 인터넷에 검색을 해보면 은 이런 디자이너 꿈 따위 되게 많이 나오거든요 근데 이제 가격 비교를 해 보면 당연히 아이에서 사는게 훨씬 싸게 쬲 뭐 품질 갖춘 경우에는 색유리 만들때 조색 할 때 쓰는 가루가 덜 옥은 그런적 땀 요정도 품질 괜찮다고 생각합니다 요 정도 사이즈의 요정도 퀄리티 야자를 많이 키우다 보니까 아 빨리 자라라 목 가지치기를 해 줄 때 나오는 잎사귀들을 요렇게 뭐 정돈되고 요런 레고 꽃 같은것도 투자 봅니다 예쁘죠 얘가 아무래도 닿는 면적에 비해서 높이가 있으니까 길고 묵직한 거는 무엇 버티 더라구요 물 없이 꽂아 나서 그런걸수도 있고 물을 조금 담고 하면 여기가 또 묵직해 지니까 저는 울 꽂이 하거나 이럴 때 쓰려고 샀습니다 잘 삽니다 마음의 추첨 이거는 제가 전에 낙성대 맛집 아따 라거 따라 소개할 때 100 빠가 쏟자 나요 예 아 갔을 때 선생님이 이렇게 귀여운 레코드판 포스터를 주시는 거에요 않는데 소장님이 주시는거 약간 c 콘 느낌 예언도 얘는 경우 쌩으로 플라스틱 외 이것도 뭐 사실 가격대비로 생각 하면 예쁘지 않게 찌 근데 얘가 괜찮을 까 이 스티커와 깡 컴 제일 멋있는 걸로 하나 해서 먹던 음료를 맞추어 같습니다 으 루거 나쁘지 않아요 귀여워요 다음 이 옴과 아 이게 뭐냐면 따란 5 얘는 물을 여기다가 담아가지고 화분을 받고 자서 누를 주는 그런겁니다 까 내가 많이 래 요 걸 해보자 놓고 이틀 정도 자리를 비워도 이렇게 천천히 물이 급수가 되는 유리 참 찬이 급수기 어 마음이 되어 있네요 굉장히 날카로워 가지고 조금 축축한 느낌이 있다고 생각했는데 다행히 마감이 되